In [78]:
from collections import defaultdict
import json
import surprise as sp
import pandas as pd
import re
us=set()
cs=set()
cin=defaultdict(str)
cni=defaultdict(str)
with open('DataCrawling/championInfo.json', 'r') as f:
    d = json.load(f)
    for champ, champ_dict in d['data'].items():
        cs.add(champ_dict['key'])
        cin[champ_dict['key']]=champ_dict['id']
        cni[champ_dict['id']]=champ_dict['key']
ul=[]
cl=[]
ml=[]
ucp=defaultdict(int)
for line in open("DataCrawling/playersInfo.txt"):
    data = line.strip().split('-')
    if len(data) != 3:
        summoner = data[-3].split(',')[-1]
        champ_list = data[-2]
        rank = data[-1]
    else:
        summoner = data[0]
        champ_list = data[1]
        rank = data[2]
    champ_list=champ_list.split(')')
    champ_list=[i.replace('(','').lstrip(',') for i in champ_list if i!='']
    sig=0
    for i in champ_list:
        if i=='': continue
        p=i.split(',')
        cs.add(p[0])
#         if int(p[1])>=7:
#             ul.append(summoner)
#             cl.append(p[0])
#             ml.append(p[1])
#             ucp[str(summoner)+'-'+str(p[0])]=p
#             sig+=2
#         elif int(p[1])>=4:
#             ul.append(summoner)
#             cl.append(p[0])
#             ml.append(p[1])
#             ucp[str(summoner)+'-'+str(p[0])]=4
#             sig+=1
        ul.append(summoner)
        cl.append(p[0])
        ml.append(p[1])
        ucp[str(summoner)+'-'+str(p[0])]=int(p[1])/7
        sig=sig+1 if int(p[1])>=4 else sig
    if sig>10:
        us.add(summoner)
    for i in cs:
        if ucp[str(summoner)+'-'+str(i)]==0:
            ul.append(summoner)
            cl.append(i)
            ml.append(0)

In [79]:
import MatrixFactorization
import numpy as np
R=np.zeros((20000,141))
us=list(us)
cs=list(cs)
itn=defaultdict(str)
for i in range(len(cs)):
    itn[i]=cin[cs[i]]
col=0
for i in us:
    if col>=20000: break
    row=0
    for j in cs:
        R[col,row]=ucp[i+'-'+j]
        row+=1
    col+=1
print(np.sum(R))

mf = MatrixFactorization.MF(R, K=20, alpha=0.1, beta=0.01, iterations=20)
mf.train()
res=mf.full_matrix()

909230.4285714283
Iteration: 10 ; error = 264.7963
Iteration: 20 ; error = 259.0223


In [93]:
res1=res[3,:]
ind=res1.argsort()[-5:][::-1]
for i in ind:
    print(itn[i])
np.set_printoptions(threshold=np.nan)
print(R[0,:])
print(res1)

Morgana
Blitzcrank
Lux
Ashe
Tristana
[0.28571429 0.         0.         0.         0.         0.14285714
 0.         0.42857143 0.14285714 0.14285714 0.         0.
 0.14285714 0.28571429 0.         0.         0.         0.
 0.14285714 0.14285714 0.         0.14285714 0.42857143 0.
 0.         0.14285714 0.         0.         0.28571429 0.28571429
 0.         0.         0.         0.         0.57142857 0.
 0.         0.28571429 0.         0.28571429 0.         0.
 0.         0.         0.         0.14285714 1.         0.14285714
 0.         1.         0.         0.71428571 0.         0.
 0.         0.         0.         0.         0.14285714 0.
 0.71428571 0.         0.42857143 0.         0.         0.85714286
 0.         0.         0.         0.         0.         0.28571429
 0.         0.         0.28571429 0.28571429 0.14285714 0.
 0.42857143 0.         0.14285714 0.         0.14285714 0.
 0.28571429 0.         1.         0.         0.14285714 0.
 1.         0.         0.         0.  